 1. Preparación de datos

In [18]:
import pandas as pd

data = pd.read_csv('data/normalized_merged_data.csv')

2. Limpieza y preprocesamiento de texto

In [19]:
import spacy

# Carga el modelo en inglés de spaCy
nlp = spacy.load('en_core_web_sm')

In [20]:
def preprocess_text(text):
    # Aplica el modelo de spaCy al texto
    doc = nlp(text)

    # Tokenización y lematización
    preprocessed_text = [token.lemma_ for token in doc if not token.is_stop]

    # Combina las palabras preprocesadas en un solo texto
    preprocessed_text = ' '.join(preprocessed_text)

    return preprocessed_text


In [21]:
data['text'] = data['text'].apply(preprocess_text)

3. Carga del modelo GPT-2

In [22]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/c1/bd/f64d67df4d3b05a460f281defe830ffab6d7940b7ca98ec085e94e024781/transformers-4.34.1-py3-none-any.whl.metadata
  Using cached transformers-4.34.1-py3-none-any.whl.metadata (121 kB)
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/5e/5d/97afbafd9d584ff1b45fcb354a479a3609bd97f912f8f1f6c563cb1fae21/filelock-3.12.4-py3-none-any.whl.metadata
  Using cached filelock-3.12.4-py3-none-any.whl.metadata (2.8 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.16.4 from https://files.pythonhosted.org/packages/ef/b5/b6107bd65fa4c96fdf00e4733e2fe5729bb9e5e09997f63074bb43d3ab28/huggingface_hub-0.18.0-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.18.0-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for pyyaml>=5.1 from https://files.pyt

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Checking for Rust toolchain....
      
      Cargo, the Rust package manager, is not installed or is not on PATH.
      This package requires Rust and Cargo to compile extensions. Install it through
      the system's package manager or via https://rustup.rs/
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer

# Carga el modelo pre-entrenado y el tokenizador
model = GPT2ForSequenceClassification.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

ImportError: cannot import name 'GPT2ForSequenceClassification' from 'transformers' (unknown location)

In [ ]:
# Tokeniza tus resúmenes
train_encodings = tokenizer(train_text, truncation=True, padding=True)
test_encodings = tokenizer(test_text, truncation=True, padding=True)

# Convierte las etiquetas a tensores (por ejemplo, usando torch)
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)


4. Entrenamiento

In [ ]:
# Añade una capa de clasificación lineal en la parte superior del modelo GPT-2
model.classifier = torch.nn.Linear(in_features=model.config.hidden_size, out_features=2)

# Entrena el modelo
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
train_dataset = torch.utils.data.TensorDataset(train_encodings.input_ids, train_encodings.attention_mask, train_labels)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

for epoch in range(epochs):
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


6. Evaluación del modelo

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Evalúa el modelo en el conjunto de prueba
with torch.no_grad():
    outputs = model(test_encodings.input_ids, attention_mask=test_encodings.attention_mask)
    predicted_labels = torch.argmax(outputs.logits, dim=1)
    accuracy = accuracy_score(test_labels, predicted_labels)
    report = classification_report(test_labels, predicted_labels)
    
print(f'Accuracy: {accuracy}')
print(report)
